In [ ]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime
import requests
from tqdm import tqdm

In [ ]:
def alerta(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.status_code
        else:
            return None
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar a URL: {e}")
        return None

# Exemplo de uso
url = "https://restfulcountries.com/api/v1"
status_code = alerta(url)
if status_code is None:
    print("O site está indisponível.")

Erro ao conectar com a API: ('Connection broken: IncompleteRead(32534 bytes read, 138644 more expected)', IncompleteRead(32534 bytes read, 138644 more expected))
Não foi possível obter os dados da API.


In [ ]:
def obter_dados_da_api(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar a API: {e}")
        return None

Falha ao acessar dados do site.


In [ ]:
def criar_dataframe(data):
    if data is None:
        return None
    
    # Extrair informações básicas dos países
    paises = []
    for pais in data['data']:
        pais_info = {
            'Nome Comum': pais['name'],
            'Nome Oficial': pais['full_name'],
            'Código Alpha-2': pais['iso2'],
            'Região': None,  # Não disponível na API
            'Latitude': None,  # Não disponível na API
            'Longitude': None,  # Não disponível na API
            'Capital': pais['capital']
        }
        paises.append(pais_info)
    
    return pd.DataFrame(paises)

In [ ]:
def criar_dataframe_populacao(data):
    if data is None:
        return None
    
    # Extrair informações demográficas e geográficas dos países
    paises = []
    for pais in data['data']:
        pais_info = {
            'Nome Comum': pais['name'],
            'População': pais['population'],
            'Área': pais['size'],
            'Continente': pais['continent'],
            'País sem Litoral': None  # Não disponível na API
        }
        paises.append(pais_info)
    
    return pd.DataFrame(paises)

In [ ]:
def criar_dataframe_membro_da_onu(data):
    if data is None:
        return None
    
    # Extrair informações sobre independência e afiliação à ONU dos países
    paises = []
    for pais in data['data']:
        pais_info = {
            'Nome Comum': pais['name'],
            'Independente': None,  # Não disponível na API
            'Membro da ONU': None,  # Não disponível na API
            'Status': None  # Não disponível na API
        }
        paises.append(pais_info)
    
    return pd.DataFrame(paises)

In [ ]:
# Obter dados da API
url = "https://restfulcountries.com/api/v1"
data = obter_dados_da_api(url)

# Criar DataFrames
df_paises = criar_dataframe(data)
df_populacao = criar_dataframe_populacao(data)
df_membro_onu = criar_dataframe_membro_da_onu(data)

# Conectar ao banco de dados SQLite
conn = sqlite3.connect('paises.db')

# Salvar DataFrames no banco de dados
df_paises.to_sql('paises', conn, if_exists='replace', index=False)
df_populacao.to_sql('populacao', conn, if_exists='replace', index=False)
df_membro_onu.to_sql('membro_onu', conn, if_exists='replace', index=False)

# Fechar a conexão
conn.close()